In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 5.1 MB/s 
     |████████████████████████████████| 197 kB 70.2 MB/s 
     |████████████████████████████████| 4.7 MB 57.6 MB/s 
     |████████████████████████████████| 120 kB 69.9 MB/s 
     |████████████████████████████████| 6.6 MB 46.9 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=80fea7a4a0f4a7384a593b49dcd2989137a323a267c3c563508ddae3235e17b4
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [ ]:
import os
os.environ["CORENLP_HOME"] = '/content/drive/MyDrive/project/stanford-corenlp-4.5.0'
import pandas


In [ ]:
import numpy as np

In [ ]:
import stanza
properties1 = {
    'openie.affinity_probability_cap': 2/3,
    'openie.triple.strict': 'true',
    'openie.resolve_coref': 'true',
    'max_entailments_per_clause': 5000,
}
# Import client module
from stanza.server import CoreNLPClient

client = CoreNLPClient(timeout=150000000, be_quiet=True, 
                       annotators=['openie'], 
endpoint='http://localhost:9011',properties=properties1)
client.start()
import time
time.sleep(10)

INFO:stanza:Writing properties to tmp file: corenlp_server-4e80bcde2eb14431.props
INFO:stanza:Starting server with command: java -Xmx5G -cp /content/drive/MyDrive/project/stanford-corenlp-4.5.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9011 -timeout 150000000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-4e80bcde2eb14431.props -annotators openie -preload -outputFormat serialized


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

doc = nlp(u"glaciers melting has a negative impact on the glaicial environment")
# words = [word.lemma_ for word in text.split()]
# text = " ".join(words)
lemma=[]

for token in doc:
    lemma.append(token.lemma_)
text = " ".join(lemma)

In [ ]:
pd_lists = pandas.read_csv('/content/drive/MyDrive/project/predicate.txt', sep="\t", header=None)
pd_lists.columns = ["predicate"]

In [ ]:
pd_lists["predicate"]

0                     have(day,number_1440_minute)
1            have(number_1_day,number_1440_minute)
2                   be_in(number_1_4_people,world)
3        be(degree_celsius,boiling_point_of_water)
4                 be(degree_celsius,boiling_point)
                           ...                    
18281               transport_through(xylem,plant)
18282                      breathe(amphibian,gill)
18283        breathe(young_amphibian,through_gill)
18284                breathe(young_amphibian,gill)
18285              breathe(amphibian,through_gill)
Name: predicate, Length: 18286, dtype: object

In [ ]:
raw_pds=pd_lists["predicate"]

In [ ]:
#Poison causes harm to a tree
test_predicates=[]
text = "A cockroach is an insect".lower()
neg=False
if text.find("not")!=-1 or text.find("n\'t")!=-1:
  neg=True
  text=text.replace("not "," ")
  text=text.replace("n't "," ")
print(text)
# print(neg)
# A being may have more of a chance of survival if it can climb up to tall places
# Albert Einstein was a German-born theoretical physicist. He developed the theory of relativity.
# You can make a telescope with a starw
# glaciers melting has a negative impact on the glaicial environment
# Using mirrors to focus collected light from heavenly bodies allows detailed observation    error
# properties1 = {
#     'openie.affinity_probability_cap': 2/3,
#     'openie.triple.strict': 'true',
#     'openie.resolve_coref': 'true',
#     # 'max_entailments_per_clause': 1,
# }
document = client.annotate(text,properties={"annotators":"tokenize,ssplit,lemma,pos,depparse,natlog,ner,coref,openie",
                                 "outputFormat": "json","triple.strict":"true","resolve_coref":"true"})
triples = []
# print(document)
for sentence in document['sentences']:
    for triple in sentence['openie']:
        triples.append({
          'subject': triple['subject'],
          'relation': triple['relation'],
            'object': triple['object']
        })
print(triples)


# be(sun,plant)
# allow(using, detailed observation)
# Poison causes harm to (A) a Tree (B) a robot (C) a house (D) a car
# poison causes harm to living things, plants are living things, Trees are plants, robots are not living things
# 
# pick 
triple_list=[]
triple_list.append(triples)
# print(triple_list)
# print(len(triple_list))
# print(len(triples))
for i in range(len(triple_list)):
  for j in range(len(triple_list[i])):
    # print(triple_list[i][j])
    # temp_triple=triple_list[i][j].copy()
    # temp_triple['subject']='asd'
    # print(temp_triple,' temp')
    # print(triple_list[i][j])
    subject=triple_list[i][j]['subject']
    subject_doc=nlp(subject)
    temp1=[]
    for token in subject_doc:
      temp1.append(token.lemma_)
      subject_lemma = " ".join(temp1)
    # print(subject_lemma,'******')
    
    relation=triple_list[i][j]['relation']
    relation_doc=nlp(relation)
    temp2=[]
    for token in relation_doc:
      temp2.append(token.lemma_)
      relation_lemma = " ".join(temp2)
    # print(relation_lemma,'******')

    obj=triple_list[i][j]['object']
    obj_doc=nlp(obj)
    temp3=[]
    for token in obj_doc:
      temp3.append(token.lemma_)
      obj_lemma = " ".join(temp3)
    # print(obj_lemma,'******')

    # print(subject,'+',relation,'+',obj)
    temp_predicate=relation_lemma+'('+subject_lemma+','+obj_lemma+')'
    # print("lemma",temp_predicate.le)
    temp_predicate=temp_predicate.replace(" ","_")
    temp_predicate=temp_predicate.lower()
    # print(temp_predicate)
    if neg==False:
      test_predicates.append(temp_predicate)
    if neg==True:
      neg_temp_predicate='not '+ temp_predicate
      test_predicates.append(neg_temp_predicate)
print(test_predicates)

a cockroach is an insect
[{'subject': 'cockroach', 'relation': 'is', 'object': 'insect'}]
['be(cockroach,insect)']


In [ ]:
QAs = pandas.read_csv('/content/drive/MyDrive/project/qtemp.tsv', sep="\t", header=0)
print(QAs)

      Unnamed: 0                                            chioceA  \
0            NaN                       Poison causes harm to a Tree   
1            0.0  The sun is responsible for puppies learning ne...   
2            1.0  When standing miles away from Mount Rushmore t...   
3            2.0  When food is reduced in the stomach the mind n...   
4            3.0                  Stars are warm lights that float    
...          ...                                                ...   
3185      3184.0  An organism that can survive without the help ...   
3186      3185.0  The nimbleness of this animal is a key adaptio...   
3187      3186.0  Birds will have different kinds of beaks depen...   
3188      3187.0                Poison causes harm to vibrant thing   
3189      3188.0                    Explosions cause harm to clouds   

                                                choiceB  \
0                         Poison causes harm to a robot   
1     The sun is responsible 

In [ ]:
index=3189
choiceA= QAs['chioceA'][index]
choiceB= QAs['choiceB'][index]
choiceC= QAs['choiceC'][index]
choiceD= QAs['choiceD'][index]
answer= QAs['answer'][index]
sep_choiceA= QAs['sep_choiceA'][index]
sep_choiceB= QAs['sep_choiceB'][index]
sep_choiceC= QAs['sep_choiceC'][index]
sep_choiceD= QAs['sep_choiceD'][index]
# correct_answer="choice"+answer
# print(correct_answer=="choiceD")

In [ ]:
choices=[choiceA,choiceB,choiceC,choiceD]
right_answer_index=ord(answer)-ord('A')
choice_doc=nlp(choices[right_answer_index])
temp3=[]
for token in choice_doc:
  temp3.append(token.lemma_)
  choice_doc_lemma = " ".join(temp3)
choice_words=choice_doc_lemma.split(' ')
print(choice_words)
sep_choices=[sep_choiceA,sep_choiceB,sep_choiceC,sep_choiceD]
sep_doc=nlp(sep_choices[right_answer_index].strip(' '))
temp4=[]
for token in sep_doc:
  temp4.append(token.lemma_)
  sep_doc_lemma = " ".join(temp4)
sep_words=sep_doc_lemma.split(' ')
print(sep_words)

['explosion', 'cause', 'harm', 'to', 'crawler']
['crawler']


In [ ]:
# question predicate
predicates=[]
choices=[choiceA,choiceB,choiceC,choiceD]

for text in choices:
  text = text.lower()
  # text="1 day has 1440 minutes"
  neg=False
  if text.find("not")!=-1 or text.find("n\'t")!=-1:
    neg=True
    text=text.replace("not "," ")
    text=text.replace("n't "," ")
  # print(neg)
  # A being may have more of a chance of survival if it can climb up to tall places
  # Albert Einstein was a German-born theoretical physicist. He developed the theory of relativity.
  # You can make a telescope with a starw
  # glaciers melting has a negative impact on the glaicial environment
  # Using mirrors to focus collected light from heavenly bodies allows detailed observation    error
  # properties1 = {
  #     'openie.affinity_probability_cap': 2/3,
  #     'openie.triple.strict': 'true',
  #     'openie.resolve_coref': 'true',
  #     # 'max_entailments_per_clause': 1,
  # }
  document = client.annotate(text,properties={"annotators":"tokenize,ssplit,lemma,pos,depparse,natlog,ner,coref,openie",
                                 "outputFormat": "json","triple.strict":"true","resolve_coref":"true"})
  triples = []
  for sentence in document['sentences']:
      for triple in sentence['openie']:
         triples.append({
            'subject': triple['subject'],
            'relation': triple['relation'],
              'object': triple['object']
          })
  # print(triples)


  # be(sun,plant)
  # allow(using, detailed observation)
  # Poison causes harm to (A) a Tree (B) a robot (C) a house (D) a car
  # poison causes harm to living things, plants are living things, Trees are plants, robots are not living things
  # pick 
  triple_list=[]
  triple_list.append(triples)
  # print(triple_list)
  # print(len(triple_list))
  # print(len(triples))
  for i in range(len(triple_list)):
    temp_pds=[]
    for j in range(len(triple_list[i])):
      
      # print(triple_list[i][j])
      # temp_triple=triple_list[i][j].copy()
      # temp_triple['subject']='asd'
      # print(temp_triple,' temp')
      # print(triple_list[i][j])
      subject=triple_list[i][j]['subject']
      subject_doc=nlp(subject)
      temp1=[]
      for token in subject_doc:
        temp1.append(token.lemma_)
        subject_lemma = " ".join(temp1)
      
      if subject_lemma[0].isdigit():
        subject_lemma="number_"+subject_lemma
    
      relation=triple_list[i][j]['relation']
      relation_doc=nlp(relation)
      temp2=[]
      for token in relation_doc:
        temp2.append(token.lemma_)
        relation_lemma = " ".join(temp2)
      if relation_lemma[0].isdigit():
        relation_lemma="number_"+relation_lemma
      # print(relation_lemma,'******')

      obj=triple_list[i][j]['object']
      obj_doc=nlp(obj)
      temp3=[]
      for token in obj_doc:
        temp3.append(token.lemma_)
        obj_lemma = " ".join(temp3)
      if obj_lemma[0].isdigit():
        obj_lemma="number_"+obj_lemma
      # print(obj_lemma,'******')

      # print(subject,'+',relation,'+',obj)
      temp_predicate=relation_lemma+'('+subject_lemma+','+obj_lemma+')'
      # print("lemma",temp_predicate.le)
      temp_predicate=temp_predicate.replace(" ","_")
      temp_predicate=temp_predicate.lower()
      # print(temp_predicate)
      if neg==False:
        temp_pds.append(temp_predicate)
      if neg==True:
        neg_temp_predicate='not '+ temp_predicate
        temp_pds.append(neg_temp_predicate)
    predicates.append(temp_pds)
  
print(predicates)


[['cause_to(explosion,cloud)', 'cause_harm_to(explosion,cloud)', 'cause(explosion,harm)'], ['cause_harm_to(explosion,radio_wave)', 'cause(explosion,harm)', 'cause_to(explosion,radio_wave)'], ['cause(explosion,harm)', 'cause_to(explosion,crawler)', 'cause_harm_to(explosion,crawler)'], ['cause(explosion,harm)', 'cause_to(explosion,airflow)', 'cause_harm_to(explosion,airflow)']]


In [ ]:
right_answer_index=ord(answer)-ord('A')
count=[]
# print(right_answer_index)
# print(predicates[right_answer_index])
for pd in predicates[right_answer_index]:
  pd_words=[]
  relation=pd.split('(')[0].replace('_',' ')
  pd_words.extend(relation.split(' '))
  # print(relation)
  # print(pd.split('('))
  arguments=pd.split('(')[1]
  subj=arguments.split(',')[0].replace('_',' ')
  pd_words.extend(subj.split(' '))
  obj=arguments.split(',')[1].strip(')').replace('_',' ')
  pd_words.extend(obj.split(' '))
  # print(subj)
  # print(obj)
  # print(pd_words)
  choice_word_set=set(choice_words)
  # print(choice_word_set)
  sep_words_set=set(sep_words)
  # print(sep_words_set)
  pd_words_set=set(pd_words)
  # print(pd_words_set & sep_words_set,'****')
  if len(pd_words_set & sep_words_set)==0:
    count.append(-1)
  else:
    count.append(len(pd_words_set & choice_word_set))
# print(count)
if len(count)==0:
  print("This problem cannot be solved currently")

#####
#modeh_index=2 ###########################################当评估特定常识时需要改
#####


In [ ]:
def get_info(answer_triple):
  neg=0
  if answer_triple.startswith('not '):
    answer_triple=answer_triple.replace('not ','')
    neg=1
    # print(neg)
  relation=answer_triple.split('(')[0]
  arguments=answer_triple.split('(')[1]
  subj=arguments.split(',')[0].replace('_',' ')
  obj=arguments.split(',')[1].strip(')').replace('_',' ')
  return relation,subj,obj,neg

# get_info("not be(robot,live_thing)")

In [ ]:
modeh_index=count.index(max(count))
# print(modeh_index)
#find 2-hop answer
#v1:- v2, v3
#v1:- v3 ,v4 
# 未处理否定的情况
answer_triple=predicates[right_answer_index][modeh_index]
# print(answer_triple)
# relation=answer_triple.split('(')[0]
# arguments=answer_triple.split('(')[1]
# subj=arguments.split(',')[0].replace('_',' ')
# obj=arguments.split(',')[1].strip(')').replace('_',' ')
relation,subj,obj,neg=get_info(answer_triple)
modeh="#modeh("+relation+"(var(nn),var(nn)))."
# print(modeh)
fact_pds=pd_lists["predicate"]
# print(len(fact_pds))
# print(fact_pds[3911])
TEM1=[]
TEM2=[]
for fact_pd in fact_pds:
  fact_relation,fact_subj,fact_obj,fact_neg=get_info(fact_pd)
  if fact_subj==subj or fact_obj==subj :
    TEM1.append(fact_pd)
  if fact_subj==obj or fact_obj==obj: 
    TEM2.append(fact_pd)
# print(TEM1)
# print(TEM2)
modeb1=[]
modeb2=[]
for tem1pd in TEM1:
  tem1pd_relation,tem1pd_subj,tem1pd_obj,tem1pd_neg=get_info(tem1pd)
  if tem1pd_subj==subj and tem1pd_obj!=obj:
    for tem2pd in TEM2:
      tem2pd_relation,tem2pd_subj,tem2pd_obj,tem2pd_neg=get_info(tem2pd)
      if tem1pd_obj==tem2pd_subj or tem1pd_obj==tem2pd_obj:
        modeb1.append(tem1pd)
        modeb2.append(tem2pd)
  if tem1pd_obj==subj and tem1pd_subj!=obj:
    for tem2pd in TEM2:
      tem2pd_relation,tem2pd_subj,tem2pd_obj,tem2pd_neg=get_info(tem2pd)
      if tem1pd_subj==tem2pd_subj or tem1pd_subj==tem2pd_obj:
        modeb1.append(tem1pd)
        modeb2.append(tem2pd)
# print(modeb1)
# print(modeb2)

# print(len(modeb1))
# print(len(modeb2))
# V1(X,Y):-V2(X,Z),V3(Z,Y)

aggragated_modeb=[]
for i in range(len(modeb1)):
  aggragated_modeb.append(modeb1[i]+' ****** '+modeb2[i])

aggragated_modeb=np.unique(aggragated_modeb)
# print('**********************************')
# print(aggragated_modeb)
u_modeb1=[]
u_modeb2=[]
for aggragated_pd in  aggragated_modeb:
  u_modeb1.append(aggragated_pd.split(' ****** ')[0])
  u_modeb2.append(aggragated_pd.split(' ****** ')[1])

# print(u_modeb1)
# print(u_modeb2)
# len(u_modeb1)

if len(u_modeb1)==0:
  print("This problem cannot be solved with proposed template rules")

This problem cannot be solved with proposed template rules


In [ ]:
for num in range(len(u_modeb1)):
  context_arguments=[]
  context_arguments.append(subj)
  context_arguments.append(obj)

  current=num   #   choose different ways
  line1_re,line1_subj,line1_obj,line1_neg=get_info(u_modeb1[current])
  line2_re,line2_subj,line2_obj,line2_neg=get_info(u_modeb2[current])

  modeb_line1="#modeb("+line1_re+"(var(nn),var(nn)))."
  modeb_line2="#modeb("+line2_re+"(var(nn),var(nn)))."
  maxv="#maxv(3)."
  max_penalty="#max_penalty(50)."
  pos="#pos({"+answer_triple+"},"
  # print(pos)
  # print(predicates)
  exclu=[]
  for i in range(4):
    if i!=right_answer_index:
      for wrong_pd in predicates[i]:
        wrong_re,wrong_subj,wrong_obj,wrong_neg=get_info(wrong_pd)
        if wrong_re==relation:
          exclu.append(wrong_pd)
          context_arguments.append(wrong_subj)
          context_arguments.append(wrong_obj)
  # print(exclu)

  exclu_line=''
  for exclu_pd in exclu:
    exclu_line+=exclu_pd+','
  exclu_line=exclu_line.strip(',')
  exclu_written="{"+exclu_line+"},"
  # exclu_written=exclu_written.strip(',')
  # print(exclu_written)
  # arguments get from exclu, predicates get from modeb search from fact predicates
  # C="{"+u_modeb1[current]
  context_res=[line1_re,line2_re]
  C=[]
  # print(context_res)
  context_arguments=np.unique(context_arguments)
  # print(context_arguments)

  for fact_pd in fact_pds:
    fact_re,fact_subj,fact_obj,fact_neg=get_info(fact_pd)
    if fact_re in context_res and (fact_subj in context_arguments or fact_obj in context_arguments):
      if fact_neg==1:
        fact_pd_temp=fact_pd.replace("not ",":- ")
        C.append(fact_pd_temp)
      else:
        C.append(fact_pd)
      
  C=np.unique(C)
  len(C)
  C_wirtten="{"
  for C_pd in C:
    C_wirtten+=C_pd+". "
  C_wirtten+="})."
  # print(C)
  file_name="ilasp_file"+str(num)+".txt"
  predicate_file=open(file_name,mode='w')
  predicate_file.write(modeh)
  predicate_file.write("\n")
  predicate_file.write(modeb_line1)
  predicate_file.write("\n")
  predicate_file.write(modeb_line2)
  predicate_file.write("\n")
  predicate_file.write(maxv)
  predicate_file.write("\n")
  predicate_file.write(max_penalty)
  predicate_file.write("\n")
  predicate_file.write(pos)
  predicate_file.write("\n")
  predicate_file.write(exclu_written)
  predicate_file.write("\n")
  predicate_file.write(C_wirtten)
  predicate_file.write("\n")
  predicate_file.close()